In [ ]:
import pandas as pd

In [2]:
import torch
print(torch.__file__)


d:\AI\MedicalChatBot\Medi_env\Lib\site-packages\torch\__init__.py


In [3]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.get_device_name(0))  # Should print your GPU name


True
NVIDIA GeForce GTX 1650


In [4]:
df =pd.read_csv('train_data_chatbot.csv')

In [5]:
import json
formatted_data = []
for _, row in df.iterrows():
    instruction = f"You are a medical assistant. Answer the following question based on the topic: {row['tags']}.\n\nQuestion: {row['short_question']}"
    formatted_data.append({
        "instruction": instruction,
        "input": "",
        "output": row["short_answer"]
    })

with open("formatted_dataset.json", "w") as f:
    for item in formatted_data:
        f.write(json.dumps(item) + "\n")

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch

dataset = load_dataset("json", data_files="formatted_dataset.json", split="train")




d:\AI\MedicalChatBot\Medi_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 47603 examples [00:00, 415301.21 examples/s]


In [ ]:
from huggingface_hub import login
login(token="YOUR_HUGGINGFACE_TOKEN")

In [8]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.52s/it]


In [9]:
model = prepare_model_for_kbit_training(model)

# Apply LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [10]:
# Tokenize the dataset
def tokenize(example):
    text = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""

    result = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize)

Map: 100%|██████████| 47603/47603 [00:43<00:00, 1096.86 examples/s]


In [11]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./llama3-medchatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)


In [12]:
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
# Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [14]:
import torch
print(torch.version.cuda)  # Should show 12.4 or compatible (e.g., 12.1)
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should show your GPU

11.8
True
NVIDIA GeForce GTX 1650


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
d:\AI\MedicalChatBot\Medi_env\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.448100
20,3.300000
30,3.300800
40,3.053300
50,3.094600
60,2.996200
70,2.686900
80,2.694700
90,2.590900
100,2.418900
